# 2022

In [92]:
import pandas as pd 
d22=pd.read_csv('data/elecciones_2022.csv', encoding='latin1')#cp1252, ISO-8859-1
d22.head(2)

,annoh,corp,eleccion,codcirc,nomdepto,coddepto,nommun,codmpio,zona,nompuesto,puesto,nomparti,codparti,nomcandi,codcandi,votos,censo,rcd___id
0,2019,3,AlcaldÃ­a,2,ANTIOQUIA,1,MEDELLIN,1,Urbano,SEC. ESC. LA ESPERANZA NO 2,1.0,PARTIDO ALIANZA VERDE,4,BEATRIZ RAVE,10,13753,12131,77aslc2o
1,2019,3,AlcaldÃ­a,2,ANTIOQUIA,1,MEDELLIN,1,Urbano,SEC. ESC. LA ESPERANZA NO 2,1.0,PARTIDO POLO DEMOCRÃTICO ALTERNATIVO,9,VICTOR JAVIER CORREA VELEZ,11,9670,12131,g1776deq


## Limpiar caracteres que no son alfanumericos en varias columnas

In [93]:
columns_to_clean=['eleccion','nomdepto','nommun','nompuesto','nomparti','nomcandi']
for column in columns_to_clean:
    d22[column] = d22[column].str.replace('[^a-zA-Z0-9]', '', regex=True)

d22.groupby(by=['annoh', 'eleccion']).size()

annoh  eleccion             
2007   Alcalda                   7890
       AsambleaDepartamental    13499
       Concejo                  10517
       Gobernacin                8264
2010   CmaraIndgena              3092
       CmaraNegritudes          18606
       CmaraTerritorial         13026
       Presidencia1eravuelta    13762
       Presidencia2davuelta      5889
       SenadoIndgena             4563
       SenadoNacional           18472
2011   Alcalda                   7712
       AsambleaDepartamental    13166
       Concejo                  11017
       Gobernacin                7531
2014   CmaraIndgena              5720
       CmaraNegritudes          14264
       CmaraTerritorial         13812
       Presidencia1eravuelta     9344
       Presidencia2davuelta      5809
       SenadoIndgena            10746
       SenadoNacional           13930
2015   Alcalda                   7797
       AsambleaDepartamental    14593
       Concejo                  12134
       Gobernacin    

## Agregar por partido

In [27]:
# d22.groupby(by=['annoh','eleccion','nompartido'], dropna=False).sum()

ag = (d22.groupby(['annoh','eleccion','nomparti'], as_index=False)
         .agg({'votos':'sum','censo':'sum'})
         .rename(columns={'annoh':'anio', 'eleccion':'elec','nomparti':'partido'}))
ag

,anio,elec,partido,votos,censo
0,2007,Alcalda,ABEJORRALVUELVEACREER,2920,12007
1,2007,Alcalda,ACCINCIUDADANAPORVILLADELEYVA,3129,7160
2,2007,Alcalda,ACCINCONELCARMEN,2295,4473
3,2007,Alcalda,ADELANTEOIBA,3672,7286
4,2007,Alcalda,ADELANTEVALLEDUPAR,32779,14786
...,...,...,...,...,...
3996,2022,SenadoNacional,PARTIDOLIBERALCOLOMBIANO,2112528,7889172
3997,2022,SenadoNacional,PARTIDONUEVOLIBERALISMO,368345,7699439
3998,2022,SenadoNacional,TarjetonesNoMarcados,530581,7884781
3999,2022,SenadoNacional,Votosenblanco,1072401,7888993


In [30]:
ag2 = (d22.groupby(['annoh','eleccion','nomparti','nomdepto','nommun'], as_index=False)
         .agg({'votos':'sum','censo':'sum'})
         .rename(columns={'annoh':'anio', 'eleccion':'elec','nomparti':'partido',
                          'nomdepto':'dpto','nommun':'mun'}))
ag2

,anio,elec,partido,dpto,mun,votos,censo
0,2007,Alcalda,ABEJORRALVUELVEACREER,ANTIOQUIA,ABEJORRAL,2920,12007
1,2007,Alcalda,ACCINCIUDADANAPORVILLADELEYVA,BOYACA,VILLADELEIVA,3129,7160
2,2007,Alcalda,ACCINCONELCARMEN,ANTIOQUIA,CARMENDEVIBORAL,2295,4473
3,2007,Alcalda,ADELANTEOIBA,SANTANDER,OIBA,3672,7286
4,2007,Alcalda,ADELANTEVALLEDUPAR,CESAR,VALLEDUPAR,32779,14786
...,...,...,...,...,...,...,...
495700,2022,SenadoNacional,Votosnulos,VAUPES,YAVARATE,11,293
495701,2022,SenadoNacional,Votosnulos,VICHADA,CUMARIBO,348,7643
495702,2022,SenadoNacional,Votosnulos,VICHADA,LAPRIMAVERA,100,5213
495703,2022,SenadoNacional,Votosnulos,VICHADA,PUERTOCARRENO,323,17359


In [31]:
ag2.to_csv('agregados/resultados_por_anio_eleccion_partido_dpto_mun.csv', index=False)

In [32]:
ag2.shape

(495705, 7)

In [37]:
import dash 
print(dash.__version__)

2.7.1


# 1.Presidencia

## a. Por anio, partido, candidato

In [38]:
d22.eleccion.value_counts()

SenadoNacional           74485
CmaraNegritudes          71420
CmaraTerritorial         56416
AsambleaDepartamental    56331
Presidencia1eravuelta    48234
Concejo                  46585
Gobernacin               33373
Alcalda                  31641
SenadoIndgena            26914
Presidencia2davuelta     23306
CmaraIndgena             19101
Plebiscitoporlapaz        4732
CmaraCITREP               2742
ConsultaAnticorrupcin     1191
Name: eleccion, dtype: int64

In [39]:
pr=d22.loc[(d22.eleccion=='Presidencia1eravuelta') | (d22.eleccion=='Presidencia2davuelta') ]
pr.head(2)

,annoh,corp,eleccion,codcirc,nomdepto,coddepto,nommun,codmpio,zona,nompuesto,puesto,nomparti,codparti,nomcandi,codcandi,votos,censo,rcd___id,elec
225689,2010,12,Presidencia1eravuelta,0,ANTIOQUIA,1,MEDELLIN,1,Urbano,SECESCLAESPERAZANo2,1.0,ALIANZASOCIALAFROCOLOMBIANAASA,1,JAIMEARAUJORENTERA,1,486,8704,rnp7di8d,Presidencia 1era vuelta
225690,2010,12,Presidencia1eravuelta,0,ANTIOQUIA,1,MEDELLIN,1,Urbano,SECESCLAESPERAZANo2,1.0,MOVIMIENTOLAVOZDELACONSCIENCIA,2,ROBINSONDEVIAGONZALEZ,2,1703,8704,1d32chab,Presidencia 1era vuelta


In [40]:
pr_ag = (pr.groupby(['annoh','eleccion', 'nomparti','nomcandi'], as_index=False)
         .agg({'votos':'sum','censo':'sum'})
         .rename(columns={'annoh':'anio', 'eleccion':'elec','nomparti':'partido'}))
pr_ag

,anio,elec,partido,nomcandi,votos,censo
0,2010,Presidencia1eravuelta,ALIANZASOCIALAFROCOLOMBIANAASA,JAIMEARAUJORENTERA,14847,6848387
1,2010,Presidencia1eravuelta,MOVIMIENTOAPERTURALIBERAL,JAIROCALDERNCARRERO,29151,7014921
2,2010,Presidencia1eravuelta,MOVIMIENTOLAVOZDELACONSCIENCIA,ROBINSONDEVIAGONZALEZ,31338,6983336
3,2010,Presidencia1eravuelta,PARTIDOCAMBIORADICAL,GERMNVARGASLLERAS,1473627,7289984
4,2010,Presidencia1eravuelta,PARTIDOCONSERVADORCOLOMBIANO,NOEMSANN,893819,7289662
...,...,...,...,...,...,...
57,2022,Presidencia2davuelta,COALICINPACTOHISTRICO,GUSTAVOPETRO,11292758,7911627
58,2022,Presidencia2davuelta,LIGADEGOBERNANTESANTICORRUPCIN,RODOLFOHERNNDEZ,10604656,7912186
59,2022,Presidencia2davuelta,VOTOSENBLANCO,VOTOSENBLANCO,500069,7912947
60,2022,Presidencia2davuelta,VOTOSNOMARCADOS,VOTOSNOMARCADOS,21461,7506523


In [41]:

pr_ag.to_csv('agregados/presidente_por_anio_eleccion_partido_candidato_mun.csv', index=False)

## b. Por Anio, Partido, Departamento

In [44]:
pr_ag2 = (pr.groupby(['annoh','eleccion', 'nomparti','nomcandi','nomdepto' ], as_index=False)
         .agg({'votos':'sum','censo':'sum'})
         .rename(columns={'annoh':'anio', 'eleccion':'elec','nomparti':'partido'}))
pr_ag2

,anio,elec,partido,nomcandi,nomdepto,votos,censo
0,2010,Presidencia1eravuelta,ALIANZASOCIALAFROCOLOMBIANAASA,JAIMEARAUJORENTERA,AMAZONAS,41,12405
1,2010,Presidencia1eravuelta,ALIANZASOCIALAFROCOLOMBIANAASA,JAIMEARAUJORENTERA,ANTIOQUIA,1677,948328
2,2010,Presidencia1eravuelta,ALIANZASOCIALAFROCOLOMBIANAASA,JAIMEARAUJORENTERA,ARAUCA,81,48663
3,2010,Presidencia1eravuelta,ALIANZASOCIALAFROCOLOMBIANAASA,JAIMEARAUJORENTERA,ARCHIPIELAGODESANANDRES,12,16163
4,2010,Presidencia1eravuelta,ALIANZASOCIALAFROCOLOMBIANAASA,JAIMEARAUJORENTERA,ATLANTICO,1606,223762
...,...,...,...,...,...,...,...
2103,2022,Presidencia2davuelta,VOTOSNULOS,VOTOSNULOS,SUCRE,3386,212610
2104,2022,Presidencia2davuelta,VOTOSNULOS,VOTOSNULOS,TOLIMA,9676,339034
2105,2022,Presidencia2davuelta,VOTOSNULOS,VOTOSNULOS,VALLEDELCAUCA,29315,322279
2106,2022,Presidencia2davuelta,VOTOSNULOS,VOTOSNULOS,VAUPES,43,17283


In [46]:
pr_ag2.to_csv('agregados/presidente_por_anio_eleccion_partido_candidato_depto.csv', index=False)

## c. Por Anio, Partido, Departamento, Municipio

In [47]:
pr_ag3 = (pr.groupby(['annoh','eleccion', 'nomparti','nomcandi','nomdepto', 'nommun'], as_index=False)
         .agg({'votos':'sum','censo':'sum'})
         .rename(columns={'annoh':'anio', 'eleccion':'elec','nomparti':'partido'}))
pr_ag3

,anio,elec,partido,nomcandi,nomdepto,nommun,votos,censo
0,2010,Presidencia1eravuelta,ALIANZASOCIALAFROCOLOMBIANAASA,JAIMEARAUJORENTERA,AMAZONAS,ELENCANTO,6,645
1,2010,Presidencia1eravuelta,ALIANZASOCIALAFROCOLOMBIANAASA,JAIMEARAUJORENTERA,AMAZONAS,LETICIA,20,5695
2,2010,Presidencia1eravuelta,ALIANZASOCIALAFROCOLOMBIANAASA,JAIMEARAUJORENTERA,AMAZONAS,MIRITIPARANA,1,359
3,2010,Presidencia1eravuelta,ALIANZASOCIALAFROCOLOMBIANAASA,JAIMEARAUJORENTERA,AMAZONAS,PUERTOALEGRIA,2,239
4,2010,Presidencia1eravuelta,ALIANZASOCIALAFROCOLOMBIANAASA,JAIMEARAUJORENTERA,AMAZONAS,PUERTOARICA,3,361
...,...,...,...,...,...,...,...,...
70781,2022,Presidencia2davuelta,VOTOSNULOS,VOTOSNULOS,VAUPES,TARAIRA,1,1160
70782,2022,Presidencia2davuelta,VOTOSNULOS,VOTOSNULOS,VICHADA,CUMARIBO,57,7517
70783,2022,Presidencia2davuelta,VOTOSNULOS,VOTOSNULOS,VICHADA,LAPRIMAVERA,24,5168
70784,2022,Presidencia2davuelta,VOTOSNULOS,VOTOSNULOS,VICHADA,PUERTOCARRENO,78,17192


# 2. Legislativa

Senado

    Nacional
            Listas,candidatos, votos por lista, votos por candidato, nulo, blanco, no marcado y censo
    Indigena
            Listas,candidatos, votos por lista, votos por candidato, nulo, blanco, no marcado y censo


Camara

    Territorial
    Indigena
    Negritude 
    CITREP

## a. Por Anio, Senado, Camara Territorial, Camara Indigenas, Camara Negritudes, Camara CITREP

## b. Por anio, senado: lista, candidatos, umbral, elegidos


## c. Por anio camara en el exterior 

### Tabla

In [53]:
cax=d22.loc[(d22.eleccion=='CmaraTerritorial') & (d22.nomdepto=='CONSULADOS') ]
cax.head(2)

,annoh,corp,eleccion,codcirc,nomdepto,coddepto,nommun,codmpio,zona,nompuesto,puesto,nomparti,codparti,nomcandi,codcandi,votos,censo,rcd___id,elec
203555,2010,2,CmaraTerritorial,1,CONSULADOS,88,ALEMANIA,120,Urbano,BerlinEmbajada,1.0,PARTIDOLIBERALCOLOMBIANO,1,PARTIDOLIBERALCOLOMBIANO,0,6,608,1nce6r88,CÃmara Territorial
203556,2010,2,CmaraTerritorial,1,CONSULADOS,88,ALEMANIA,120,Urbano,BerlinEmbajada,1.0,PARTIDOCONSERVADORCOLOMBIANO,2,OSCARJOSEPEREZALVAREZ,401,4,608,nk7av6l7,CÃmara Territorial


In [72]:
cax_ag = (cax.groupby(['annoh', 'nomparti'], as_index=False)
         .agg({'votos':'sum','censo':'sum'})
         .rename(columns={'annoh':'anio', 'nomparti':'partido'}))
cax_ag.head(2)

,anio,partido,votos,censo
0,2010,ALIANZASOCIALINDIGENA,377,92023
1,2010,MOVIMIENTOAPERTURALIBERAL,86,58771


### Grafica

In [90]:
import plotly.express as px
import plotly.graph_objects as go
fig = go.Figure(data=go.Heatmap(x=cax_ag.anio, y=cax_ag.partido,z=cax_ag.votos))
fig.show()

## d. Por anio camara en el exterior: USA

### Tabla

In [85]:
us=cax.loc[( cax.nommun=='ESTADOSUNIDOS') ]
us.head(2)

,annoh,corp,eleccion,codcirc,nomdepto,coddepto,nommun,codmpio,zona,nompuesto,puesto,nomparti,codparti,nomcandi,codcandi,votos,censo,rcd___id,elec
203752,2010,2,CmaraTerritorial,1,CONSULADOS,88,ESTADOSUNIDOS,360,Urbano,AtlantaConsulado,2.0,PARTIDOLIBERALCOLOMBIANO,1,PARTIDOLIBERALCOLOMBIANO,0,381,5907,bx1d235t,CÃmara Territorial
203753,2010,2,CmaraTerritorial,1,CONSULADOS,88,ESTADOSUNIDOS,360,Urbano,AtlantaConsulado,2.0,PARTIDOCONSERVADORCOLOMBIANO,2,PARTIDOCONSERVADORCOLOMBIANO,0,526,5907,esgc93dw,CÃmara Territorial


### Grafica

In [91]:
us_ag = (us.groupby(['annoh', 'nomparti','nommun','nompuesto'], as_index=False)
         .agg({'votos':'sum','censo':'sum'})
         .rename(columns={'annoh':'anio', 'nompuesto':'nompuesto'}))
us_ag.head(2)

,anio,nomparti,nommun,nompuesto,votos,censo
0,2010,ALIANZASOCIALINDIGENA,ESTADOSUNIDOS,AtlantaConsulado,97,5907
1,2010,MOVIMIENTOAPERTURALIBERAL,ESTADOSUNIDOS,AtlantaConsulado,22,5907


In [89]:
us_fig = go.Figure(data=go.Heatmap(x=us_ag.anio, y=us_ag.nompuesto,z=us_ag.votos))
us_fig.show()

# 3. Departamentales

Gobernacion

Alcaldia

Asamblea

Concejo


# 2018

In [2]:
import pandas as pd 
ca=pd.read_stata('data/2018_Camara.dta')
ca.head()

,id_electoral,ano,tipo_eleccion,fecha_eleccion,coddpto,departamento,codmpio,municipio,circunscripcion,codigo_partido,codigo_lista,primer_apellido,segundo_apellido,nombres,votos,curules
0,320180001,2018.0,Cámara de Representantes,Marzo 11,5,ANTIOQUIA,5001,MEDELLÍN,Departamental,20180005.0,0.0,,,PARTIDO SOMOS,2331.0,0.0
1,320180036,2018.0,Cámara de Representantes,Marzo 11,5,ANTIOQUIA,5001,MEDELLÍN,Afrodescendientes,20100037.0,0.0,,,FUNDACIÓN DEPORTIVA VIAFARA FUNDEVIA,40.0,0.0
2,320180036,2018.0,Cámara de Representantes,Marzo 11,5,ANTIOQUIA,5001,MEDELLÍN,Afrodescendientes,20180013.0,0.0,,,C. C. DE LA COMUN. NEGRA DE TIERRA BAJA MI TER...,33.0,0.0
3,320180036,2018.0,Cámara de Representantes,Marzo 11,5,ANTIOQUIA,5001,MEDELLÍN,Afrodescendientes,20180039.0,0.0,,,KUSUTO MAGENDE COKUMALU DE LURUACO,93.0,0.0
4,320180036,2018.0,Cámara de Representantes,Marzo 11,5,ANTIOQUIA,5001,MEDELLÍN,Afrodescendientes,20180023.0,0.0,,,CONSEJO COMUNITARIO DE COMUNIDADES NEGRAS SANT...,24.0,0.0


In [4]:
ca.shape

(110125, 16)

In [5]:
ca.columns

Index(['id_electoral', 'ano', 'tipo_eleccion', 'fecha_eleccion', 'coddpto',
       'departamento', 'codmpio', 'municipio', 'circunscripcion',
       'codigo_partido', 'codigo_lista', 'primer_apellido', 'segundo_apellido',
       'nombres', 'votos', 'curules'],
      dtype='object')

In [3]:
ca.circunscripcion.value_counts()

Departamental             72713
Afrodescendientes         27768
Indígena                   7101
Especial Internacional     2543
Name: circunscripcion, dtype: int64

In [10]:
co=ca.codigo_lista.value_counts()

In [ ]:
ca.segundo_apellido.value_counts()

In [13]:
ca3=ca.loc[ (ca.circunscripcion=='Departamental')  ]
ca4=ca3.loc[(ca3.codigo_lista==994) | (ca3.codigo_lista==997) |(ca3.codigo_lista==998) |(ca3.codigo_lista==999) ]


In [14]:
import plotly.express as px
fig= px.bar(ca4, x='codigo_lista', y='votos')
fig.show()